In [2]:
import sys
sys.path.append('../')

In [3]:
import AbreivBot.Functions as functions
import AbreivBot.Models.BaseModels as base_model

2023-04-15 04:21:39.664962: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas as pd
import numpy as np
import string

In [5]:
import tensorflow as tf
devices = tf.config.list_physical_devices()

2023-04-15 04:21:41.643141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-15 04:21:41.648903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-15 04:21:41.649407: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
devices

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
abbrev = pd.read_csv('TrainAndTest/traindata/cleaned_abbreviations.csv')
abbrev = abbrev[abbrev.columns[1:]]

In [8]:
abbrev['parse_ylabel'] = abbrev.apply(lambda x: ''.join(functions.encode_ylabel(x['Abbreviation'], x['Word'])), axis=1)
abbrev['WordLength'] = abbrev['Word'].apply(lambda x: len(x))

In [9]:
abbrev.head()

,Abbreviation,Word,parse_ylabel,WordLength
0,A.D.,Anno Domini,A.@@@D.@@@@,11
1,A.V.,Authorized Version,A.@@@@@@@@@V.@@@@@,18
2,Abbrev.,abbreviation,Abbrev.@@@@@,12
3,Abbrev.,abbreviations,Abbrev.@@@@@@,13
4,Abd.,Aberdeen,Ab@@d.@@,8


In [10]:
possible_chars = string.ascii_letters + string.digits + ' .@'
possible_chars

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .@'

In [11]:
model = base_model.BaseModel(possible_chars)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, None, len(possible_chars)))

2023-04-15 04:21:42.902013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-15 04:21:42.902218: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-15 04:21:42.902337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
train = abbrev.sample(frac=0.8, random_state=1337)
test = abbrev.drop(train.index)

In [13]:
import AbreivBot.TrainModel as train_model
import AbreivBot.TestModel as test_model

In [14]:
history = train_model.train(model, train, possible_chars, verbose=0)

2023-04-15 04:21:44.915798: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-15 04:21:44.919099: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-15 04:21:44.921516: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [15]:
model.summary()

Model: "base_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  65        
                                                                 
 flatten (Flatten)           multiple                  0 (unused)
                                                                 
 lstm (LSTM)                 multiple                  34060     
                                                                 
 dense (Dense)               multiple                  4290      
                                                                 
Total params: 38,415
Trainable params: 38,415
Non-trainable params: 0
_________________________________________________________________


In [16]:
results = test_model.test(model, test, possible_chars, verbose=0)

2023-04-15 04:23:05.899878: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,8,65]
	 [[{{node Placeholder/_1}}]]
2023-04-15 04:23:06.106182: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [21,13,65]
	 [[{{node Placeholder/_1}}]]
2023-04-15 04:23:06.214007: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [

In [17]:
results

{8: [2.1696033477783203, 0.48046875],
 13: [1.8304523229599, 0.5604395866394043],
 10: [1.8789170980453491, 0.5285714268684387],
 11: [2.358142614364624, 0.445141077041626],
 14: [2.172107696533203, 0.5158730149269104],
 9: [2.2505667209625244, 0.43518519401550293],
 12: [2.1966841220855713, 0.5080645084381104],
 15: [2.371595621109009, 0.4333333373069763],
 7: [2.5516579151153564, 0.3891625702381134],
 16: [2.6288716793060303, 0.40625],
 5: [2.736447811126709, 0.4888888895511627],
 20: [3.0673153400421143, 0.6000000238418579],
 19: [1.9535748958587646, 0.5789473652839661],
 6: [2.4509613513946533, 0.4444444477558136],
 4: [2.3277764320373535, 0.5],
 22: [1.68913996219635, 0.7424242496490479],
 18: [2.6735541820526123, 0.3333333432674408],
 17: [1.9571533203125, 0.47058823704719543],
 21: [1.4287668466567993, 0.761904776096344]}

In [20]:
model.save_weights('Models/TrainedModels/BaseModel')